# Bixi Telemetry Data Warehouse
### Data Engineering Capstone Project

#### Project Summary
The goal of this project is to build a data warehouse for Bixi, a Montreal-based bike-sharing service. The modeled data will then be used to analyze ride pattern and availability of bikes at every station, enabling Bixi to optimize replenishment activities.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import requests
import io
import zipfile
import json
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import explode
from pyspark.sql import functions as F
from pyspark.sql import types as T
import psycopg2
import sql_queries
import configparser

In [2]:
GBFS_SOURCE = "s3a://bixi-gbfs-data"
GBFS_DESTINATION = "s3a://bixi-gbfs-data-parquet"

STATION_INFO_DATA = "https://gbfs.velobixi.com/gbfs/en/station_information.json"
TRIP_APRIL_DATA = "https://sitewebbixi.s3.amazonaws.com/uploads/docs/biximontreal-rentals-2021-04-87aaed.zip"
TRIP_APRIL_DATA_FILE = "OD_2021_04.csv"

### Step 1: Scope the Project and Gather Data

#### Scope 

This project will outline the steps to obtain the raw data, transform and clean it, write it back to an S3 bucket and finally injest it into a Redshift datawarehouse. The source data will be collected from GBFS feed data from Bixi that had been previously extracted every 5 minutes for several weeks and stored in an S3 bucket. The latest station metadata and historical trip data will be retrieved from the Bixi REST API directly. The data will be read, cleansed and transfered using a Spark cluster, for efficiency given the large data size. The data warehouse will be hosted on an Amazon Redshift cluster.

#### Describe and Gather Data 
The main dataset used will be a json output of the Bixi API, which exposes their [GBFS (General Bikeshare Feed Specification)](https://github.com/NABSA/gbfs) feed. The API was queries every 5 minutes for 3 weeks, using a Python script scheduled by a cron job. The response json was then uploaded to an Amazon S3 bucket.

#### Setting up the Spark cluster and reading in the data

We start by provisioning the AWS access ID and secret key in the spark config and instantiate a spark session

In [3]:
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get("default", "aws_access_key_id") 
access_key = config.get("default", "aws_secret_access_key")

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH","DWH_PORT")
DWH_ENDPOINT = config.get("DWH","DWH_ENDPOINT")
DWH_IAM_ROLE_NAME = config.get("DWH", "DWH_IAM_ROLE_NAME")

In [4]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
conf.set("fs.s3a.access.key", access_id)
conf.set("fs.s3a.secret.key", access_key)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

Once the Spark session is created, all the `json` files in the S3 bucket are then read into a Spark dataframe. The nested station data is exploded to individual rows and split by column. The unix epoch timestamp in column `last_updated` is converted to a timestamp, then several time parts are expanded into multiple column, which will later be used to partition the parquet output 

In [5]:
df = spark.read.json(f"{GBFS_SOURCE}/20210413022503.json")

In [6]:
df_flat = (df
    .select("last_updated",explode("data.stations").alias('stations'))
    .select("last_updated", "stations.*")    
)

In [7]:
df_flat = df_flat \
    .withColumn("last_updated_dt", F.from_unixtime("last_updated").cast('timestamp')) \
    .withColumn("station_id", df_flat.station_id.cast('long')) \
    .withColumn("year", F.year("last_updated_dt")) \
    .withColumn("month", F.month("last_updated_dt")) \
    .withColumn("day", F.dayofmonth("last_updated_dt"))


In [8]:
df_flat = df_flat.select(
    ["station_id"] + [c for c in df_flat.columns if c not in [
        'station_id','eightd_has_available_keys','last_updated'
    ]]
)

Next, the Bixi API is called again to obtain the latest station metadata

In [9]:
json_output = "data/station_info.json"
with open(json_output, 'w') as f:
    json.dump(requests.get(STATION_INFO_DATA).json(),f)

In [10]:
df_station = spark.read.json(json_output)

In [11]:
df_station_flat = df_station \
    .select(explode("data.stations").alias('stations')) \
    .select("stations.station_id"
        , "stations.name"
        , 'stations.lat'
        , "stations.lon"
    )

The last dataset is trip data from the month april 2021, which is the same timeframe as the GBFS feed, which needs to be extracted from a `zip` file, read into a spark dataframe from a `csv` file, enriched with time based columns, and finally written back to S3 as a `parquet` file, partitioned by year, month, day, similar to the GBFS feed.

In [12]:
csv_path = f"data/{TRIP_APRIL_DATA_FILE}"

r = requests.get(TRIP_APRIL_DATA)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extract(TRIP_APRIL_DATA_FILE, path = 'data')

'data/OD_2021_04.csv'

In [13]:
df_trips = spark.read.csv(csv_path, header=True)
df_trips = df_trips.withColumn("trip_id",F.monotonically_increasing_id())

In [14]:
df_trips = df_trips \
    .withColumn("start_date", F.from_utc_timestamp("start_date","EST")) \
    .withColumn("end_date", F.from_utc_timestamp("end_date","EST")) \
    .withColumn("duration_sec", df_trips.duration_sec.cast('int')) \
    .withColumn("start_station_id", df_trips.emplacement_pk_start.cast('long')) \
    .withColumn("end_station_id", df_trips.emplacement_pk_end.cast('long')) \
    .withColumn("year", F.year("start_date")) \
    .withColumn("month", F.month("start_date")) \
    .withColumn("day", F.dayofmonth("start_date"))

df_trips = df_trips.select(["trip_id"] + [c for c in df_trips.columns if c not in ["trip_id","emplacement_pk_start","emplacement_pk_end"]])

### Step 2: Explore and Assess the Data

Since this data comes from the Bixi API and comes from automated telemetry, we can assume that the data is faily clean. Neverthless there are likely scenarios we would like to exclude.

Let's first look like there are any trip data with 0 duration, missing station, or where the end datetime is smaller or equal to the start datetime. 

In [15]:
errors = df_trips.where(
    df_trips.end_date <= df_trips.start_date
).collect()

print(f"{str(len(errors))} errors found")

0 errors found


In [16]:
zero_duration = df_trips.where(
    df_trips.duration_sec == 0
).collect()

print(f"{str(len(zero_duration))} errors found")

0 errors found


In [17]:
missing_station = df_trips.where(
    df_trips.start_station_id.isNull() | \
    df_trips.end_station_id.isNull()
).collect()

print(f"{str(len(missing_station))} errors found")

0 errors found


No errors found there. We can also check if we have trips that started and ended at the same station. Since this could be a valid use case for someone going on a ride for fun, we can restrict to a duration of no more than 2 min. Such trips are likely not real trips and should be excluded.

In [18]:
dummy_trip = df_trips.where(
    (df_trips.start_station_id == df_trips.end_station_id) & \
    (df_trips.duration_sec < 120)
)

print(f"{str(len(dummy_trip.collect()))} errors found")

778 errors found


There are many of such instances. Let us remove them from the data frame

In [19]:
df_trips_clean = df_trips.join(dummy_trip, on='trip_id', how='left_anti')

Lastly, the transformed and cleaned dataframe are written back to an S3 bucket as `parquet` and `csv` files

In [ ]:
gbfs_file_destination = f"{GBFS_DESTINATION}/gbfs.parquet"

df_flat.write \
    .partitionBy('year','month','day') \
    .parquet(path = gbfs_file_destination, mode = "overwrite") 

In [ ]:
stations_destination = f"{GBFS_DESTINATION}/stations.csv"
df_station_flat.write.csv(
    path = stations_destination, 
    mode = "overwrite"
)

In [21]:
trips_destination = f"{GBFS_DESTINATION}/trips.parquet"
df_trips_clean.write \
    .partitionBy('year','month', 'day') \
    .parquet(
        path = trips_destination, 
        mode = "overwrite"
    )

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [22]:
conn = psycopg2.connect(
    dbname= DWH_DB, 
    host= DWH_ENDPOINT, 
    port= DWH_PORT, 
    user= DWH_DB_USER, 
    password= DWH_DB_PASSWORD
)

In [23]:
def run_query(query):
    cur = conn.cursor()
    try:
        cur.execute(query)
        try:
            results = cur.fetchall()
        except:
            results = None
        conn.commit()
        cur.close()
    except Exception as e:
        print(e)
        cur.close()
    return results

In [29]:
for query in sql_queries.create_queries:
    run_query(query)

In [24]:
trips_destination = f"{GBFS_DESTINATION}/trips.parquet"

copy_trips = sql_queries.copy_table_query.format(
    'trips',trips_destination.replace('s3a','s3'), access_id, access_key, 'parquet'
)
run_query(copy_trips)

In [31]:
stations_destination = f"{GBFS_DESTINATION}/stations.csv"
copy_stations = sql_queries.copy_table_query.format(
    'stations',stations_destination.replace('s3a','s3'), access_id, access_key, 'csv'
)
run_query(copy_stations)

In [32]:
gbfs_file_destination = f"{GBFS_DESTINATION}/gbfs.parquet"
copy_gbfs= sql_queries.copy_table_query.format(
    'gbfs',gbfs_file_destination.replace('s3a','s3'), access_id, access_key, 'parquet'
)
run_query(copy_gbfs)

#### 4.2 Data Quality Checks

Next, we will ensure the pipeline ran sucessfully by comparing the row count in the source dataframe with the row count of the resulting table. An error will be thrown for any errors

In [25]:
#Data Check for table gbfs
df_count = df_flat.count()
tbl_count = run_query(sql_queries.check_row_count.format("gbfs"))[0][0]
if df_count != tbl_count:
    raise ValueError ("Mismatch of row count between parquet file and table gbfs")

In [26]:
#Data Check for table stations
df_count = df_station_flat.count()
tbl_count = run_query(sql_queries.check_row_count.format("stations"))[0][0]
if df_count != tbl_count:
    raise ValueError (f"Mismatch of row count between parquet file and table stations. Expected {df_count}, got {tbl_count}")

In [27]:
#Data Check for table trips
df_count = df_trips_clean.count()
tbl_count = run_query(sql_queries.check_row_count.format("trips"))[0][0]
if df_count != tbl_count:
    raise ValueError (f"Mismatch of row count between parquet file and table trips. Expected {df_count}, got {tbl_count}")

#### 4.3 Data dictionary 

`trips`

| Column | Data Type | Description |
|:--- |:--- |:---|
| trip_id | **integer** | Unique identifier for the trips, added during transformation|
| start_date | **timestamp** | Start datetime of the trip|
| end_date | **timestamp** | End datetime of the trip|
| duration_sec | **integer** | Trip duration in seconds|
| is_member | **boolean** | Flag identifying if trip was made by a Bixi member|
| start_station_id | **integer** | Station identifier where trip began
| end_startion_id | **integer** | Station identifier where trip ended

`station`

| Column | Data Type | Description |
|:--- |:--- |:---|
| station_id | **integer** | Unique identifier for the station|
| name | **varchar** | Long name of the station|
| lat | **decimal** | Latitude of the station location|
| lon | **decimal** | Longitude of the station location|

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

 Given the nature of the data, this pipeline should be run every day.

In [2]:
1+1

2